In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup

import requests
import re
import pandas as pd
import time

ModuleNotFoundError: No module named 'selenium'

In [44]:
상폐기업뉴스 = pd.read_csv('../news_preprocess/상폐기업뉴스전처리후.csv')
상폐기업뉴스.drop('Unnamed: 0', axis=1, inplace=True)

pattern1 = r'\([^)]*\)'
pattern2 = r"\[([^]]+)\]"

상폐기업뉴스['뉴스기사본문전처리'] = [re.sub(pattern1, '', s) for s in 상폐기업뉴스['뉴스기사본문']]
상폐기업뉴스['뉴스기사본문전처리'] = [re.sub(pattern2, '', s) for s in 상폐기업뉴스['뉴스기사본문전처리']]
상폐기업뉴스['뉴스기사본문전처리'] = [re.sub('[^/^$|\s+/가-힣\.]', '', s) for s in 상폐기업뉴스['뉴스기사본문전처리']]
상폐기업뉴스['뉴스기사본문전처리'] = [re.sub('[[ \s]{2,}\.{2,}]', '', s) for s in 상폐기업뉴스['뉴스기사본문전처리']]
상폐기업뉴스['뉴스기사본문전처리'] = [re.sub('\n', '', s) for s in 상폐기업뉴스['뉴스기사본문전처리']]
상폐기업뉴스['뉴스기사본문전처리'] = [re.sub('[/+]', '', s) for s in 상폐기업뉴스['뉴스기사본문전처리']]
상폐기업뉴스.drop('뉴스기사본문', axis=1, inplace=True)
상폐기업뉴스

,기업,기사발행일,기사제목,뉴스기사본문전처리
0,네이쳐글로벌,2010.08.16. 오후 5:21,"네이쳐글로벌, 반기검토의견 ""의견거절'…거래정지",네이쳐글로벌는 일 반기 검토 의견이 의견거절로 관리종목 지정 사유가 발생했다고 공시...
1,네이쳐글로벌,2010.07.15. 오후 6:46,네이쳐글로벌 신용등급 `CC`로 두단계 하향,네이쳐글로벌의 신용등급을 기존 에서 으로 하향 조정한다고 한신정평가가 일 밝혔다....
2,네이쳐글로벌,2010.06.03. 오후 5:06,"네이쳐글로벌, 20대 1 감자 실시",네이쳐글로벌은 대 감자를 결의했다고 일 공시했다.감자 주주총회일은 다음달 일 감...
3,네이쳐글로벌,2010.05.24. 오전 10:41,<생생코스닥> 네이쳐글로벌 3거래일 연속 하한가,네이쳐글로벌이 연 거래일 연속 하한가를 기록하며 주가가 원 선 마저 위협 받고 있다...
4,네이쳐글로벌,2010.05.18. 오후 1:23,"네이쳐글로벌 ""생수시장 진출 하반기 150억 계획""",네이쳐글로벌은 대규모 생수회사를 자산양수도 방식으로 인수 추진 중이라고 일 밝혔다....
...,...,...,...,...
2695,코썬바이오,2020.01.09. 오전 10:07,"코썬바이오, 상한가 진입... 이평선 역배열 상황에서 반등 시도",코썬바이오일 오전 코썬바이오가 상한가를 기록했다. 코썬바이오는 균형생식환 황찬고 등...
2696,코썬바이오,2020.03.04. 오후 5:45,"코썬바이오, 거래소에 개선계획서 제출",한국거래소 코스닥시장본부는 일 상장적격성 실질심사 대상으로 결정된 코썬바이오가 개...
2697,코썬바이오,2020.01.10. 오후 6:42,코썬바이오 거래정지,한국거래소는 코스닥 상장기업인 코썬바이오에 대해 월 일부터 상장적격성 실질심사 대상...
2698,코썬바이오,2019.12.27. 오후 6:13,"코썬바이오, 10억원 규모 유상증자 결정",코썬바이오는 운영자금 억원을 조달하기 위해 제자배정증자 방식의 유상증자를 결정했다...


In [56]:
driver = webdriver.Chrome(r"C:\Users\dbswo\OneDrive\Desktop\논문복원\크롤링\chromedriver.exe")

url = "https://papago.naver.com/"
driver.get(url)
기사번역 = []

for 뉴스 in 상폐기업뉴스['뉴스기사본문전처리']:

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "lxml")

    time.sleep(1)

    input = driver.find_element_by_xpath('//*[@id="txtSource"]')
    input.clear()
    input.send_keys(뉴스)

    button = driver.find_element_by_xpath('//*[@id="btnTranslate"]')
    button.click()
    time.sleep(2)
    try:

        result = driver.find_element_by_xpath('//*[@id="txtTarget"]/span').text

        if result != 기사번역[-1]:
            기사번역.append(result)

        else:
            raise Exception('번역이 늦어 중복입니다')

    except Exception:

        try:
            input = driver.find_element_by_xpath('//*[@id="txtSource"]')
            input.clear()
            input.send_keys(뉴스)

            button = driver.find_element_by_xpath('//*[@id="btnTranslate"]')
            button.click()
            time.sleep(2)

            result = driver.find_element_by_xpath('//*[@id="txtTarget"]/span').text

            if result != 기사번역[-1]:
                기사번역.append(result)

            else:
                raise Exception('번역이 늦어 중복입니다')
        
        except:

            input = driver.find_element_by_xpath('//*[@id="txtSource"]')
            input.clear()
            input.send_keys(뉴스)

            button = driver.find_element_by_xpath('//*[@id="btnTranslate"]')
            button.click()
            time.sleep(6)

            result = driver.find_element_by_xpath('//*[@id="txtTarget"]/span').text
            기사번역.append(result)

상폐기업뉴스['기사번역']  = 기사번역
상폐기업뉴스

C:\Users\dbswo\AppData\Local\Temp\ipykernel_8584\2001320575.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\dbswo\OneDrive\Desktop\논문복원\크롤링\chromedriver.exe")
C:\Users\dbswo\AppData\Local\Temp\ipykernel_8584\2001320575.py:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  input = driver.find_element_by_xpath('//*[@id="txtSource"]')
C:\Users\dbswo\AppData\Local\Temp\ipykernel_8584\2001320575.py:18: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = driver.find_element_by_xpath('//*[@id="btnTranslate"]')
C:\Users\dbswo\AppData\Local\Temp\ipykernel_8584\2001320575.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  result = driver.find_element_by_xpath('//*[@id="txtTarget"]/span').text
C:\Us

KeyboardInterrupt: 

In [54]:
len(기사번역)

241

In [55]:
기사번역

["Nature Global announced that there was a reason for designating a management item due to the rejection of opinions on the review in the first half. In response, the Korea Exchange announced that it would suspend trading of Nature Global stock certificates.Reporter Choi Sung-nam of Hankyung Dotcom. Today's recommended event is a surprise explosion. The habit of winning a free trial of the value of the highest secret power. Do not reprint or redistribute Hankyung Dotcom. Copyright under the Korea Online Newspaper Association's Digital News Usage Rules",
 "A credit rating agency said it will lower Nature Global's credit rating from the previous one.The rating downgrade was explained by considering the possibility of high uncertainty in new businesses and possible realization of contingent liabilities related to litigation due to the weakening of the main business of lowering creditworthiness to financial institutions.Hanshin Ratings said it negatively evaluated its rating outlook in the